In [1]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
import openpyxl
import akshare as ak
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 5000
pd.options.display.float_format = lambda x: '%.5f' % x
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.sans-serif'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False
%config InlineBackend.figure_format='svg'
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

In [2]:
def excess_profit(code):
    # 去除单引号
    code = code.strip("'")
    
    try:
        # 获取股票数据
        price = ak.stock_zh_a_hist(symbol=code, period='daily', adjust='hfq', 
                                  start_date='20221218', end_date='20250101')
        
        # 检查是否获取到数据
        if price.empty:
            print(f"警告: 股票 {code} 没有历史数据")
            return 0.0, 0.0, 0.0

        
        # 检查关键列是否存在
        required_columns = ['日期', '开盘', '收盘', '最高', '最低']
        missing_columns = [col for col in required_columns if col not in price.columns]
        
        if missing_columns:
            print(f"错误: 股票 {code} 缺少必要的列: {missing_columns}")
            return 0.0, 0.0, 0.0
        
        # 数据预处理
        price = price.dropna()
        
        # 确保列名正确
        if '收盘' in price.columns:
            price['MA10'] = price['收盘'].rolling(window=10).mean()
        else:
            print(f"错误: 股票 {code} 没有找到 '收盘' 列")
            return 0.0, 0.0, 0.0
        
        # 删除前10行(因为MA10前10行是NaN)
        price = price.drop(index=range(10)).reset_index(drop=True)
        
        # 检查处理后的数据是否足够
        if len(price) < 100:
            print(f"错误: 股票 {code} 数据不足，无法进行回测")
            return 0.0, 0.0, 0.0
        
        # 初始化策略参数
        initial_cash = 10000000.0
        tax = 0.00025
        
        # 创建结果DataFrame
        df = pd.DataFrame(columns=['日期', '开盘', '收盘', 'MA10', '涨跌幅', '可买', '持仓', '持有天数', '交易信号', '总市值', '现金', '策略收益率', '股票收益率'],
                          index=range(len(price)))
        
        # 复制数据
        df[['日期', '开盘', '收盘', 'MA10']] = price[['日期', '开盘', '收盘', 'MA10']]
        
        # 初始化列
        df['持仓'] = 0
        df['总市值'] = 0.0
        df['现金'] = initial_cash
        df['策略收益率'] = 0.0
        df['股票收益率'] = 0.0
        df['交易信号'] = 0
        df['持有天数'] = 0
        df['总资产'] = initial_cash
        df['可买'] = 0
        
        # 执行策略回测
        for i in range(1,len(df)):
            # 计算可买股数
            df.loc[i, '可买'] = (((df.loc[i-1, '现金'] / df.loc[i, '开盘'])*(1-tax)) // 100) * 100

                    # 金叉死叉生成交易信号
            if df.loc[i-1,'MA10']>=df.loc[i-1,'收盘'] and df.loc[i,'MA10']<df.loc[i,'收盘']:
                df.loc[i,'交易信号']=1
            if df.loc[i-1,'MA10']<=df.loc[i-1,'收盘'] and df.loc[i,'MA10']>df.loc[i,'收盘']:
                df.loc[i,'交易信号']=-1


                    # 计算交易成本
            buy = 0
            sell = 0

            if i < len(df):
                buy = max(df.loc[i, '开盘'] * df.loc[i, '可买'] * tax, 5) + df.loc[i, '开盘'] * df.loc[i, '可买']

            if i in range(len(df)) and df.loc[i-1, '持仓'] > 0:
                sell = df.loc[i, '开盘'] * df.loc[i-1, '持仓'] - max(df.loc[i, '开盘'] * df.loc[i-1, '持仓'] * tax, 5)

                    # 第二天开盘执行交易
            if df.loc[i-1, '交易信号'] == 1 and df.loc[i-1, '现金'] >= buy and df.loc[i-1, '持仓'] == 0 and i!=len(df)-1:
                df.loc[i, '现金'] = df.loc[i-1, '现金'] - buy
                df.loc[i, '持仓'] = df.loc[i-1, '持仓'] + df.loc[i, '可买']
            elif df.loc[i-1, '交易信号'] == -1 and df.loc[i-1, '持仓'] > 0:
                df.loc[i, '现金'] = df.loc[i-1, '现金'] + sell
                df.loc[i, '持仓'] = 0
            else:
                df.loc[i, '现金'] = df.loc[i-1, '现金']
                df.loc[i, '持仓'] = df.loc[i-1, '持仓']

            # 计算总资产和收益率
            df.loc[i, '总市值'] = df.loc[i, '持仓'] * df.loc[i, '收盘']
            df.loc[i, '总资产'] = df.loc[i, '总市值'] + df.loc[i, '现金']
            df.loc[i, '策略收益率'] = (df.loc[i, '总资产'] / initial_cash) - 1
            df.loc[i, '股票收益率'] = (df.loc[i, '收盘'] / df.loc[0, '收盘']) - 1
        
            # 计算最终结果
        if len(df) > 0:
            excess = df.loc[len(df)-1, '策略收益率'] - df.loc[len(df)-1, '股票收益率']
            strategy = df.loc[len(df)-1, '策略收益率']
            stock = df.loc[len(df)-1, '股票收益率']

            # 结果检测
            if not all([isinstance(x, (int, float)) for x in [excess, strategy, stock]]):
                print(f"错误: 股票 {code} 的收益率计算结果包含非数值类型")
                return 0.0, 0.0, 0.0

            if any(pd.isna([excess, strategy, stock])):
                print(f"错误: 股票 {code} 的收益率计算结果包含NaN值")
                return 0.0, 0.0, 0.0

            if any([abs(x) > 10 for x in [excess, strategy, stock]]):
                print(f"警告: 股票 {code} 的收益率计算结果异常大: "
                          f"超额收益率={excess:.4f}, 策略收益率={strategy:.4f}, 股票收益率={stock:.4f}")

                # 打印最终结果
            print(f"股票 {code} 策略回测完成: "
                      f"超额收益率={excess:.4f}, 策略收益率={strategy:.4f}, 股票收益率={stock:.4f}")

            return excess, strategy, stock
        else:
            print(f"错误: 股票 {code} 的回测结果DataFrame为空")
            return 0.0, 0.0, 0.0
            
    except Exception as e:
        print(f"处理股票 {code} 时出错: {e}")
        return 0.0, 0.0, 0.0

In [3]:
# 获取A股实时数据
df_excess = ak.stock_zh_a_spot()

# 移除代码中的前缀
df_excess['代码'] = df_excess['代码'].str[2:]
# 筛选沪深市股票
df_excess = df_excess[~df_excess['代码'].str.startswith(('4','8','9'))]
df_excess = df_excess[~df_excess['名称'].str.startswith(('ST','*ST'))]

# 重置索引
df_excess = df_excess.reset_index(drop=True)

# 删除不需要的列
columns_to_drop = ['最新价', '涨跌额', '涨跌幅', '买入', '卖出', '昨收', '今开', '最高', '最低', '成交量', '成交额', '时间戳']
df_excess = df_excess.drop(columns=[col for col in columns_to_drop if col in df_excess.columns])

# 应用策略并填充结果
def calculate_returns(row):
    code = row['代码']
    return pd.Series(excess_profit(code), index=['超额收益率', '策略收益率', '股票收益率'])

# 先创建新的列，再应用函数
df_excess[['超额收益率', '策略收益率', '股票收益率']] = df_excess.apply(calculate_returns, axis=1)

# 过滤掉三个收益率都为0的行
df_excess = df_excess[~((df_excess['超额收益率'] == 0) & 
                        (df_excess['策略收益率'] == 0) & 
                        (df_excess['股票收益率'] == 0))]

# 删除包含缺失值的行
df_excess = df_excess.dropna()

# 按超额收益率降序排列
#df_excess = df_excess.sort_values(by='超额收益率', ascending=False)
#df_excess=df_excess.reset_index(drop=True)

# 将收益率转换为百分比格式
#df_excess[['超额收益率', '策略收益率', '股票收益率']] = df_excess[['超额收益率', '策略收益率', '股票收益率']].apply(lambda x: x * 100).applymap("{:.2f}%".format)

# 保存结果到Excel文件
df_excess.to_excel(r'E:/jupyter/excel/MA10策略回测.xlsx')

# 显示结果
df_excess

Please wait for a moment:   0%|          | 0/68 [00:00<?, ?it/s]

股票 600000 策略回测完成: 超额收益率=-0.0639, 策略收益率=0.2521, 股票收益率=0.3160
股票 600004 策略回测完成: 超额收益率=0.1316, 策略收益率=-0.1466, 股票收益率=-0.2782
股票 600006 策略回测完成: 超额收益率=-0.2243, 策略收益率=0.0198, 股票收益率=0.2440
股票 600007 策略回测完成: 超额收益率=-0.4420, 策略收益率=0.0487, 股票收益率=0.4907
股票 600008 策略回测完成: 超额收益率=-0.2443, 策略收益率=-0.0694, 股票收益率=0.1749
股票 600009 策略回测完成: 超额收益率=0.1175, 策略收益率=-0.2352, 股票收益率=-0.3527
股票 600010 策略回测完成: 超额收益率=0.0729, 策略收益率=0.0416, 股票收益率=-0.0313
股票 600011 策略回测完成: 超额收益率=-0.0059, 策略收益率=-0.0820, 股票收益率=-0.0760
股票 600012 策略回测完成: 超额收益率=-0.8425, 策略收益率=0.1410, 股票收益率=0.9836
股票 600015 策略回测完成: 超额收益率=-0.3243, 策略收益率=0.0951, 股票收益率=0.4194
股票 600016 策略回测完成: 超额收益率=-0.0595, 策略收益率=0.1429, 股票收益率=0.2024
股票 600017 策略回测完成: 超额收益率=-0.1052, 策略收益率=0.0457, 股票收益率=0.1508
股票 600018 策略回测完成: 超额收益率=-0.2442, 策略收益率=-0.0983, 股票收益率=0.1460
股票 600019 策略回测完成: 超额收益率=-0.1486, 策略收益率=0.0223, 股票收益率=0.1709
股票 600020 策略回测完成: 超额收益率=-0.2647, 策略收益率=0.0635, 股票收益率=0.3282
股票 600021 策略回测完成: 超额收益率=0.0227, 策略收益率=-0.0677, 股票收益率=-0.0904
股票 600022 策略回测完成: 超额收益率=0.1893, 策

D:\anaconda3\Lib\site-packages\jupyter_client\session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


股票 600123 策略回测完成: 超额收益率=-0.1732, 策略收益率=-0.1942, 股票收益率=-0.0210
股票 600125 策略回测完成: 超额收益率=-0.2748, 策略收益率=-0.1999, 股票收益率=0.0749
股票 600126 策略回测完成: 超额收益率=0.0179, 策略收益率=0.1520, 股票收益率=0.1341
股票 600127 策略回测完成: 超额收益率=-0.0470, 策略收益率=-0.1735, 股票收益率=-0.1265
股票 600128 策略回测完成: 超额收益率=0.5189, 策略收益率=0.5454, 股票收益率=0.0265
股票 600129 策略回测完成: 超额收益率=0.5612, 策略收益率=0.3535, 股票收益率=-0.2077
股票 600131 策略回测完成: 超额收益率=-0.2492, 策略收益率=-0.0582, 股票收益率=0.1911
股票 600132 策略回测完成: 超额收益率=0.3131, 策略收益率=-0.0932, 股票收益率=-0.4063
股票 600133 策略回测完成: 超额收益率=1.1885, 策略收益率=2.0127, 股票收益率=0.8243
股票 600135 策略回测完成: 超额收益率=0.2147, 策略收益率=0.1032, 股票收益率=-0.1115
股票 600137 策略回测完成: 超额收益率=-0.0169, 策略收益率=-0.0654, 股票收益率=-0.0485
股票 600138 策略回测完成: 超额收益率=-0.0739, 策略收益率=-0.4019, 股票收益率=-0.3280
股票 600141 策略回测完成: 超额收益率=0.4869, 策略收益率=0.3048, 股票收益率=-0.1821
股票 600143 策略回测完成: 超额收益率=0.0924, 策略收益率=0.0187, 股票收益率=-0.0737
股票 600148 策略回测完成: 超额收益率=-0.2217, 策略收益率=0.0840, 股票收益率=0.3057
股票 600149 策略回测完成: 超额收益率=0.2592, 策略收益率=-0.0995, 股票收益率=-0.3587
股票 600150 策略回测完成: 超额收益率=-0.7486

D:\anaconda3\Lib\site-packages\jupyter_client\session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


...

股票 301251 策略回测完成: 超额收益率=0.0104, 策略收益率=-0.2319, 股票收益率=-0.2423
股票 301252 策略回测完成: 超额收益率=-0.5211, 策略收益率=-0.3709, 股票收益率=0.1502
股票 301255 策略回测完成: 超额收益率=-0.6408, 策略收益率=-0.1456, 股票收益率=0.4952
股票 301256 策略回测完成: 超额收益率=0.4270, 策略收益率=0.3070, 股票收益率=-0.1200
股票 301257 策略回测完成: 超额收益率=-0.1320, 策略收益率=-0.4284, 股票收益率=-0.2964
股票 301258 策略回测完成: 超额收益率=0.1181, 策略收益率=-0.2608, 股票收益率=-0.3789
股票 301259 策略回测完成: 超额收益率=-1.2384, 策略收益率=0.9443, 股票收益率=2.1826
股票 301260 策略回测完成: 超额收益率=0.6908, 策略收益率=0.0326, 股票收益率=-0.6582
股票 301261 策略回测完成: 超额收益率=-0.3194, 策略收益率=-0.3271, 股票收益率=-0.0078
股票 301262 策略回测完成: 超额收益率=-0.0088, 策略收益率=-0.0752, 股票收益率=-0.0664
股票 301263 策略回测完成: 超额收益率=0.0827, 策略收益率=-0.0807, 股票收益率=-0.1634
股票 301265 策略回测完成: 超额收益率=0.0633, 策略收益率=-0.1980, 股票收益率=-0.2613
股票 301266 策略回测完成: 超额收益率=0.5524, 策略收益率=0.0040, 股票收益率=-0.5484


D:\anaconda3\Lib\site-packages\jupyter_client\session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


股票 301267 策略回测完成: 超额收益率=0.2852, 策略收益率=-0.3033, 股票收益率=-0.5885
股票 301268 策略回测完成: 超额收益率=0.0489, 策略收益率=-0.6563, 股票收益率=-0.7051
股票 301269 策略回测完成: 超额收益率=0.0023, 策略收益率=0.3007, 股票收益率=0.2984
股票 301270 策略回测完成: 超额收益率=-0.3391, 策略收益率=-0.1964, 股票收益率=0.1427
股票 301272 策略回测完成: 超额收益率=0.0931, 策略收益率=-0.3976, 股票收益率=-0.4907
股票 301273 策略回测完成: 超额收益率=0.2093, 策略收益率=-0.3491, 股票收益率=-0.5584
警告: 股票 301275 没有历史数据
股票 301276 策略回测完成: 超额收益率=0.0928, 策略收益率=-0.0009, 股票收益率=-0.0937
股票 301277 策略回测完成: 超额收益率=0.0972, 策略收益率=0.1049, 股票收益率=0.0077
股票 301278 策略回测完成: 超额收益率=-0.0004, 策略收益率=-0.5882, 股票收益率=-0.5878
股票 301279 策略回测完成: 超额收益率=-0.0759, 策略收益率=-0.1451, 股票收益率=-0.0692
股票 301280 策略回测完成: 超额收益率=-0.6766, 策略收益率=-0.3381, 股票收益率=0.3385
股票 301281 策略回测完成: 超额收益率=-0.0600, 策略收益率=0.1210, 股票收益率=0.1809
股票 301282 策略回测完成: 超额收益率=-0.0105, 策略收益率=-0.2068, 股票收益率=-0.1964
股票 301283 策略回测完成: 超额收益率=0.2332, 策略收益率=-0.3271, 股票收益率=-0.5603
股票 301285 策略回测完成: 超额收益率=-1.2584, 策略收益率=-0.1251, 股票收益率=1.1333
股票 301286 策略回测完成: 超额收益率=-0.5511, 策略收益率=-0.1253, 股票收益率=0.4259
股票 30

股票 301510 策略回测完成: 超额收益率=0.1638, 策略收益率=-0.2794, 股票收益率=-0.4431
股票 301511 策略回测完成: 超额收益率=-0.0911, 策略收益率=-0.5676, 股票收益率=-0.4766
股票 301512 策略回测完成: 超额收益率=0.3159, 策略收益率=-0.0662, 股票收益率=-0.3821
股票 301515 策略回测完成: 超额收益率=0.2931, 策略收益率=-0.1875, 股票收益率=-0.4806
股票 301516 策略回测完成: 超额收益率=-0.3784, 策略收益率=-0.4508, 股票收益率=-0.0724
股票 301517 策略回测完成: 超额收益率=0.2800, 策略收益率=0.0229, 股票收益率=-0.2571
股票 301518 策略回测完成: 超额收益率=0.2519, 策略收益率=-0.2042, 股票收益率=-0.4562
股票 301519 策略回测完成: 超额收益率=0.1796, 策略收益率=-0.3906, 股票收益率=-0.5702
股票 301520 策略回测完成: 超额收益率=0.2757, 策略收益率=-0.1094, 股票收益率=-0.3851
错误: 股票 301522 数据不足，无法进行回测
股票 301525 策略回测完成: 超额收益率=0.1648, 策略收益率=-0.2753, 股票收益率=-0.4401
股票 301526 策略回测完成: 超额收益率=0.0156, 策略收益率=-0.3265, 股票收益率=-0.3422
股票 301528 策略回测完成: 超额收益率=0.3174, 策略收益率=-0.2658, 股票收益率=-0.5831
股票 301529 策略回测完成: 超额收益率=-0.0242, 策略收益率=-0.3858, 股票收益率=-0.3616
股票 301533 策略回测完成: 超额收益率=0.2416, 策略收益率=-0.1236, 股票收益率=-0.3652
警告: 股票 301535 没有历史数据
股票 301536 策略回测完成: 超额收益率=-0.7687, 策略收益率=0.4804, 股票收益率=1.2491
股票 301538 策略回测完成: 超额收益率=-0.0275, 策略收益

,代码,名称,超额收益率,策略收益率,股票收益率
0,600000,浦发银行,-0.06388,0.25211,0.31599
1,600004,白云机场,0.13165,-0.14657,-0.27822
2,600006,东风股份,-0.22426,0.01976,0.24402
3,600007,中国国贸,-0.44203,0.04870,0.49074
4,600008,首创环保,-0.24430,-0.06936,0.17495
5,600009,上海机场,0.11751,-0.23515,-0.35266
6,600010,包钢股份,0.07289,0.04157,-0.03132
7,600011,华能国际,-0.00594,-0.08196,-0.07602
8,600012,皖通高速,-0.84254,0.14103,0.98357
9,600015,华夏银行,-0.32432,0.09505,0.41937


#